In [ ]:
import datetime
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.formatting.rule import CellIsRule

# Diccionario con tipos de proceso y su plazo (días hábiles)
TIPOS_PROCESO = {
    "Ordinario": 20,
    "Verbal": 10,
    "Verbal Sumario": 5,
    "Ejecutivo Singular": 10,
    "Monitorio": 10,
    "Jurisdicción Voluntaria": 5
}

# Conjunto para almacenar procesos vencidos
procesos_vencidos = set()

# Lista para almacenar los procesos
procesos = []

# Clase ProcesoJudicial (POO aplicada)
class ProcesoJudicial:
    def __init__(self, nombre, tipo, fecha_inicio):
        self.nombre = nombre.title()
        self.tipo = tipo
        self.fecha_inicio = fecha_inicio
        self.plazo_dias = TIPOS_PROCESO.get(tipo, 5)
        self.fecha_limite = self.sumar_dias_habiles(self.fecha_inicio, self.plazo_dias)
        self.notificado = False

    def sumar_dias_habiles(self, fecha, dias):
        contador = 0
        while contador < dias:
            fecha += datetime.timedelta(days=1)
            if fecha.weekday() < 5:
                contador += 1
        return fecha

    def mostrar_resumen(self):
        estado = " EN PLAZO" if self.fecha_limite >= datetime.datetime.today() else " VENCIDO"
        print(f"{self.nombre} ({self.tipo}):")
        print(f"  - Inicio: {self.fecha_inicio.date()}")
        print(f"  - Límite: {self.fecha_limite.date()} ({self.plazo_dias} días hábiles)")
        print(f"  - Estado: {estado}\n")

    def verificar_alerta(self):
        hoy = datetime.datetime.today()
        dias_restantes = (self.fecha_limite.date() - hoy.date()).days
        if dias_restantes < 0:
            print(f" '{self.nombre}' está vencido desde hace {-dias_restantes} días.")
            procesos_vencidos.add(self.nombre)
        elif dias_restantes <= 3 and not self.notificado:
            print(f"  ¡Alerta! '{self.nombre}' vence en {dias_restantes} días.")
            self.notificado = True
        else:
            print(f"  '{self.nombre}' está al día ({dias_restantes} días restantes).")

# Función para mostrar tipos de procesos con fecha de vencimiento
def mostrar_tipos_disponibles(fecha_base):
    print("\n Tipos de Procesos (plazos hábiles y fechas límite):")
    for i, (tipo, dias) in enumerate(TIPOS_PROCESO.items(), 1):
        vencimiento = ProcesoJudicial("Ejemplo", tipo, fecha_base).fecha_limite
        print(f"{i}. {tipo} ({dias} días) - Vence: {vencimiento.date()}")

# Función para agregar nuevo proceso
def agregar_proceso():
    try:
        fecha_input = input("Fecha de inicio (YYYY-MM-DD) o Enter para usar hoy: ").strip()
        fecha_base = datetime.datetime.today() if fecha_input == "" else datetime.datetime.strptime(fecha_input, "%Y-%m-%d")
        mostrar_tipos_disponibles(fecha_base)
        seleccion = int(input("\nSeleccione el número del tipo de proceso: "))
        tipo = list(TIPOS_PROCESO.keys())[seleccion - 1]
        nombre = input("Nombre del proceso: ")
        proceso = ProcesoJudicial(nombre, tipo, fecha_base)
        procesos.append(proceso)
        print(f"\n Proceso '{nombre}' registrado con vencimiento el {proceso.fecha_limite.date()}.")
    except ValueError:
        print(" Formato de fecha incorrecto. Use AAAA-MM-DD.")
    except IndexError:
        print(" Selección inválida. Ingrese un número correcto del menú.")
    except Exception as e:
        print(f" Error inesperado: {e}")

# Función para ver todos los procesos
def ver_cronograma():
    if not procesos:
        print(" No hay procesos registrados.")
    else:
        print("\n Cronograma de Procesos Judiciales:")
        for proceso in procesos:
            proceso.mostrar_resumen()

# Función para verificar alertas
def verificar_alertas():
    if not procesos:
        print(" No hay procesos para verificar.")
        return
    print("\n Verificando vencimientos:")
    for proceso in procesos:
        proceso.verificar_alerta()
    if procesos_vencidos:
        print("\n Procesos vencidos:")
        for p in procesos_vencidos:
            print(f" - {p}")

# Función para filtrar por rango de fechas
def filtrar_por_rango():
    try:
        inicio = datetime.datetime.strptime(input("Fecha de inicio (YYYY-MM-DD): ").strip(), "%Y-%m-%d")
        fin = datetime.datetime.strptime(input("Fecha de fin (YYYY-MM-DD): ").strip(), "%Y-%m-%d")
        print(f"\nProcesos con vencimiento entre {inicio.date()} y {fin.date()}:")
        encontrados = False
        for p in procesos:
            if inicio.date() <= p.fecha_limite.date() <= fin.date():
                p.mostrar_resumen()
                encontrados = True
        if not encontrados:
            print(" No se encontraron procesos en ese rango.")
    except Exception as e:
        print(f" Error en las fechas: {e}")

# Función de resumen estadístico
def resumen_general():
    total = len(procesos)
    vencidos = len(procesos_vencidos)
    activos = total - vencidos
    print("\n Resumen General:")
    print(f" Total de procesos: {total}")
    print(f" En plazo: {activos}")
    print(f" Vencidos: {vencidos}")
    tipos = {tipo: 0 for tipo in TIPOS_PROCESO}
    for p in procesos:
        tipos[p.tipo] += 1
    print("\n Tipos registrados:")
    for tipo, cantidad in tipos.items():
        if cantidad:
            print(f" - {tipo}: {cantidad}")

# FUNCIÓN MEJORADA para guardar en Excel con formato atractivo
def guardar_en_excel(nombre_archivo="procesos_judiciales.xlsx"):
    if not procesos:
        print("No hay procesos para guardar.")
        return

    # Preparar datos
    data = []
    hoy = datetime.datetime.today()

    for p in procesos:
        dias_restantes = (p.fecha_limite.date() - hoy.date()).days

        if dias_restantes < 0:
            estado = "VENCIDO"
            urgencia = "🔴 CRÍTICO"
        elif dias_restantes <= 3:
            estado = "PRÓXIMO A VENCER"
            urgencia = "🟡 URGENTE"
        elif dias_restantes <= 7:
            estado = "EN PLAZO"
            urgencia = "🟠 ATENCIÓN"
        else:
            estado = "EN PLAZO"
            urgencia = "🟢 NORMAL"

        data.append({
            "📋 Nombre del Proceso": p.nombre,
            "⚖️ Tipo de Proceso": p.tipo,
            "📅 Fecha de Inicio": p.fecha_inicio.date(),
            "⏰ Fecha Límite": p.fecha_limite.date(),
            "📊 Días Hábiles": p.plazo_dias,
            "📈 Estado": estado,
            "🚨 Nivel de Urgencia": urgencia,
            "⏳ Días Restantes": dias_restantes if dias_restantes >= 0 else f"Vencido hace {abs(dias_restantes)} días"
        })

    # Crear DataFrame
    df = pd.DataFrame(data)

    # Guardar el archivo Excel básico
    df.to_excel(nombre_archivo, index=False)

    # Abrir el archivo para aplicar formato
    wb = load_workbook(nombre_archivo)
    ws = wb.active

    # Configurar el título de la hoja
    ws.title = "Control de Procesos Judiciales"

    # Definir colores y estilos
    header_fill = PatternFill(start_color="2E4053", end_color="2E4053", fill_type="solid")
    header_font = Font(color="FFFFFF", bold=True, size=12)

    # Colores para estados
    vencido_fill = PatternFill(start_color="E74C3C", end_color="E74C3C", fill_type="solid")
    urgente_fill = PatternFill(start_color="F39C12", end_color="F39C12", fill_type="solid")
    atencion_fill = PatternFill(start_color="FF8C00", end_color="FF8C00", fill_type="solid")
    normal_fill = PatternFill(start_color="27AE60", end_color="27AE60", fill_type="solid")

    # Fuentes para contenido
    content_font = Font(size=10)
    bold_font = Font(bold=True, size=10)

    # Configurar bordes
    thin_border = Border(
        left=Side(style='thin'),
        right=Side(style='thin'),
        top=Side(style='thin'),
        bottom=Side(style='thin')
    )

    # Aplicar formato a encabezados
    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = Alignment(horizontal='center', vertical='center')
        cell.border = thin_border

    # Aplicar formato a contenido y colores según estado
    for row_num, row in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row), 2):
        for cell in row:
            cell.font = content_font
            cell.border = thin_border
            cell.alignment = Alignment(horizontal='center', vertical='center')

        # Colorear filas según el nivel de urgencia
        urgencia_cell = ws[f'G{row_num}']  # Columna de Nivel de Urgencia

        if "🔴 CRÍTICO" in str(urgencia_cell.value):
            for cell in row:
                cell.fill = vencido_fill
                cell.font = Font(color="FFFFFF", bold=True, size=10)
        elif "🟡 URGENTE" in str(urgencia_cell.value):
            for cell in row:
                cell.fill = urgente_fill
                cell.font = Font(color="FFFFFF", bold=True, size=10)
        elif "🟠 ATENCIÓN" in str(urgencia_cell.value):
            for cell in row:
                cell.fill = atencion_fill
                cell.font = Font(color="FFFFFF", bold=True, size=10)
        else:
            for cell in row:
                cell.fill = normal_fill
                cell.font = Font(color="FFFFFF", bold=True, size=10)

    # Ajustar ancho de columnas
    column_widths = {
        'A': 30,  # Nombre del Proceso
        'B': 25,  # Tipo de Proceso
        'C': 15,  # Fecha de Inicio
        'D': 15,  # Fecha Límite
        'E': 12,  # Días Hábiles
        'F': 20,  # Estado
        'G': 18,  # Nivel de Urgencia
        'H': 20   # Días Restantes
    }

    for column, width in column_widths.items():
        ws.column_dimensions[column].width = width

    # Agregar una hoja de resumen
    ws_resumen = wb.create_sheet("📊 Resumen Estadístico")

    # Calcular estadísticas
    total_procesos = len(procesos)
    procesos_vencidos_count = len([p for p in procesos if (p.fecha_limite.date() - hoy.date()).days < 0])
    procesos_urgentes = len([p for p in procesos if 0 <= (p.fecha_limite.date() - hoy.date()).days <= 3])
    procesos_normales = total_procesos - procesos_vencidos_count - procesos_urgentes

    # Datos del resumen
    resumen_data = [
        ["📊 RESUMEN ESTADÍSTICO DE PROCESOS JUDICIALES", ""],
        ["", ""],
        ["📈 Total de Procesos:", total_procesos],
        ["🔴 Procesos Vencidos:", procesos_vencidos_count],
        ["🟡 Procesos Urgentes (≤3 días):", procesos_urgentes],
        ["🟢 Procesos Normales:", procesos_normales],
        ["", ""],
        ["📅 Fecha de Generación:", hoy.strftime("%d/%m/%Y %H:%M")],
        ["", ""],
        ["🏛️ DISTRIBUCIÓN POR TIPO DE PROCESO:", ""]
    ]

    # Agregar distribución por tipo
    tipos_count = {}
    for p in procesos:
        tipos_count[p.tipo] = tipos_count.get(p.tipo, 0) + 1

    for tipo, cantidad in tipos_count.items():
        resumen_data.append([f"⚖️ {tipo}:", cantidad])

    # Escribir datos del resumen
    for row_num, (label, value) in enumerate(resumen_data, 1):
        ws_resumen[f'A{row_num}'] = label
        ws_resumen[f'B{row_num}'] = value

        # Formato especial para el título
        if row_num == 1:
            ws_resumen[f'A{row_num}'].font = Font(size=16, bold=True, color="2E4053")
            ws_resumen.merge_cells(f'A{row_num}:B{row_num}')
            ws_resumen[f'A{row_num}'].alignment = Alignment(horizontal='center')

        # Formato para encabezados de sección
        elif "DISTRIBUCIÓN" in str(label) or any(x in str(label) for x in ["📈", "📅", "🏛️"]):
            ws_resumen[f'A{row_num}'].font = Font(size=12, bold=True, color="34495E")

        # Formato para datos
        else:
            ws_resumen[f'A{row_num}'].font = Font(size=11, bold=True)
            ws_resumen[f'B{row_num}'].font = Font(size=11)

    # Ajustar columnas del resumen
    ws_resumen.column_dimensions['A'].width = 40
    ws_resumen.column_dimensions['B'].width = 15

    # Guardar el archivo
    wb.save(nombre_archivo)

    ruta_absoluta = os.path.abspath(nombre_archivo)
    print(f"\n✅ Procesos guardados exitosamente en '{nombre_archivo}'")
    print(f"📂 Ruta del archivo: {ruta_absoluta}")
    if procesos_vencidos:
        print(f"⚠️ También se guardaron los procesos vencidos (marcados como 'VENCIDO').")

# Menú interactivo
def menu():
    print("\n MODO RESUMEN AUTOMÁTICO")
    resumen_general()
    verificar_alertas()

    while True:
        print("\n\n===== MENÚ PRINCIPAL =====")
        print("1. Registrar nuevo proceso")
        print("2. Ver cronograma")
        print("3. Ver alertas de vencimiento")
        print("4. Ver resumen general")
        print("5. Filtrar procesos por rango de fechas")
        print("6. Salir y guardar en Excel")

        opcion = input("Seleccione una opción: ")
        if opcion == "1":
            agregar_proceso()
        elif opcion == "2":
            ver_cronograma()
        elif opcion == "3":
            verificar_alertas()
        elif opcion == "4":
            resumen_general()
        elif opcion == "5":
            filtrar_por_rango()
        elif opcion == "6":
            guardar_en_excel()
            print("\n🔚 El sistema ha finalizado. ¡Gracias por usar la herramienta de gestión judicial!")
            break
        else:
            print(" Opción inválida. Intente de nuevo.")

# Ejecutar el programa
menu()
from google.colab import files
files.download("procesos_judiciales.xlsx")


 MODO RESUMEN AUTOMÁTICO

 Resumen General:
 Total de procesos: 0
 En plazo: 0
 Vencidos: 0

 Tipos registrados:
 No hay procesos para verificar.


===== MENÚ PRINCIPAL =====
1. Registrar nuevo proceso
2. Ver cronograma
3. Ver alertas de vencimiento
4. Ver resumen general
5. Filtrar procesos por rango de fechas
6. Salir y guardar en Excel
